In [101]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [102]:
# # Data inladen
df = pd.read_csv("final_df.csv")
print(df.head())

# # Apply One-Hot Encoding for the 'binary_trainingtype' column
# df = pd.get_dummies(df, columns=['binary_trainingtype'], drop_first=False)

# # Convert boolean columns to 0 and 1
# df[df.columns[df.columns.str.startswith('binary_trainingtype')]] = df[df.columns[df.columns.str.startswith('binary_trainingtype')]].astype(int)

# # Display the updated DataFrame
# print(df.head())

   ervaring  500_split  2k tijd  binary_trainingtype  binary_geslacht  \
0       1.0      104.6    379.9                    0                0   
1       1.0      104.7    379.9                    0                0   
2       1.0      104.3    379.9                    0                0   
3       1.0      104.0    379.9                    0                0   
4       1.0      104.1    379.9                    0                0   

   binary_gewichtsklasse  
0                      1  
1                      1  
2                      1  
3                      1  
4                      1  


In [103]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.neural_network import MLPRegressor

# # Prepare the data
# df_X = df.drop(columns=['2k tijd'])
# y = df['2k tijd']

# # Normalize the data
# sc = MinMaxScaler()
# norm_df_X = sc.fit_transform(df_X)
# norm_y = sc.fit_transform(y.values.reshape(-1, 1))

# # Split the data into training and validation sets
# train_X, valid_X, train_y, valid_y = train_test_split(norm_df_X, norm_y, test_size=0.3, random_state=1)

# # Define the parameter grid for GridSearchCV
# param_grid = {
#     'hidden_layer_sizes': [(64, 32), (128, 64, 32), (128, 64)],
#     'activation': ['relu'],
#     'solver': ['adam'],
#     'learning_rate_init': [0.001, 0.01],
#     'max_iter': [2000, 3000]
# }

# # Initialize GridSearchCV with the MLPRegressor and the parameter grid
# grid_search = GridSearchCV(estimator=MLPRegressor(random_state=1), 
#                            param_grid=param_grid, 
#                            cv=5,  # 5-fold cross-validation
#                            n_jobs=-1,  # Use all available CPU cores
#                            verbose=2)  # Set to 1 or 2 for more output details

# # Fit GridSearchCV on the training data
# grid_search.fit(train_X, train_y.ravel())

# # Get the best parameters and score from GridSearchCV
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print(f"Best Parameters: {best_params}")
# print(f"Best Cross-validation Score: {best_score}")

# # Get the best model from the grid search
# best_model = grid_search.best_estimator_

# # Evaluate the best model on the validation data
# validation_score = best_model.score(valid_X, valid_y)
# print(f"Validation Score: {validation_score}")


In [104]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import GridSearchCV

# df_X = df.drop(columns=['2k tijd'])
# y = df['2k tijd']

# sc = MinMaxScaler()
# norm_df_X = sc.fit_transform(df_X)
# norm_y = sc.fit_transform(y.values.reshape(-1, 1))

# train_X, valid_X, train_y, valid_y = train_test_split(norm_df_X, norm_y, test_size=0.3, random_state=1)

# car_lm = MLPRegressor(hidden_layer_sizes=(64,32), 
#                       activation='relu',
#                       solver='adam',
#                       random_state=1,
#                       max_iter=3000)
# car_lm.fit(train_X, train_y.ravel())
# car_lm.score(valid_X, valid_y)

In [105]:
# param_grid = {
#     'hidden_layer_sizes': [(64,32), (128, 64)],
#     'solver': ['adam', 'lbfgs'],
#     'activation': ['relu'],
#     'max_iter': [3000, 5000]
# }

# gridSearch = GridSearchCV(MLPRegressor(), param_grid, 
#                           cv=5, n_jobs=-1,
#                           return_train_score=True)
# gridSearch.fit(train_X, train_y.ravel())

# print('Best score: ', gridSearch.best_score_,
#       '\nBest parameters: ', gridSearch.best_params_)

# display=['param_hidden_layer_sizes', 'param_solver', 'param_max_iter', 'mean_test_score', 'mean_train_score']
# print(pd.DataFrame(gridSearch.cv_results_)[display])

In [106]:
from sklearn.model_selection import train_test_split

# Eerst de data opschudden om bias te voorkomen
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Zorg ervoor dat elke ervaring, geslacht en gewichtsklasse in elke set vertegenwoordigd zijn
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

for ervaring in df['ervaring'].unique():
    for geslacht in df['binary_geslacht'].unique():
        for gewichtsklasse in df['binary_gewichtsklasse'].unique():
            subset = df[(df['ervaring'] == ervaring) & (df['binary_geslacht'] == geslacht) & (df['binary_gewichtsklasse'] == gewichtsklasse)]
            if not subset.empty:
                temp_train, temp_temp = train_test_split(subset, test_size=0.3, random_state=42)
                temp_val, temp_test = train_test_split(temp_temp, test_size=0.3, random_state=42)
                train_data = pd.concat([train_data, temp_train])
                val_data = pd.concat([val_data, temp_val])
                test_data = pd.concat([test_data, temp_test])

# Reset indexen
train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# Controleren op juiste verdeling
print(f"Trainingsdata: {len(train_data)} rijen")
print(f"Validatiedata: {len(val_data)} rijen")
print(f"Testdata: {len(test_data)} rijen")

Trainingsdata: 3018 rijen
Validatiedata: 905 rijen
Testdata: 391 rijen


In [107]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Separate features (X) and target (y) for training, validation, and test sets
X_train = train_data.drop(columns=['2k tijd'])
y_train = train_data['2k tijd']
X_val = val_data.drop(columns=['2k tijd'])
y_val = val_data['2k tijd']
X_test = test_data.drop(columns=['2k tijd'])
y_test = test_data['2k tijd']

# Standardize the features
scaler = StandardScaler()
scaler.fit(X_train)  # Fit on training data only
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Create and train the MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=2000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Validate the model
y_val_pred = mlp.predict(X_val_scaled)
val_mse = mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation MSE: {val_mse:.2f}")
print(f"Validation R^2: {val_r2:.2f}")

# Evaluate on test data
y_test_pred = mlp.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Test MSE: {test_mse:.2f}")
print(f"Test R^2: {test_r2:.2f}")

# Get the predicted values on the test set
y_test_pred = mlp.predict(X_test_scaled)

# Create a DataFrame to compare the actual vs predicted values
results = pd.DataFrame({
    'Actual 2k tijd': y_test,
    'Predicted 2k tijd': y_test_pred,
    'Difference': y_test - y_test_pred
})

# Display a few examples
print(results.head(10))  # You can adjust the number of rows displayed


Validation MSE: 101.69
Validation R^2: 0.90
Test MSE: 98.40
Test R^2: 0.90
   Actual 2k tijd  Predicted 2k tijd  Difference
0           427.7         421.711274    5.988726
1           420.5         426.018609   -5.518609
2           418.6         418.302867    0.297133
3           427.7         425.243480    2.456520
4           427.7         428.465421   -0.765421
5           420.5         425.553682   -5.053682
6           421.9         422.408890   -0.508890
7           449.0         424.147627   24.852373
8           431.6         425.773892    5.826108
9           431.6         423.913581    7.686419
